In [1]:
import igraph as ig
import pandas as pd
import numpy as np
from sklearn.metrics import adjusted_mutual_info_score as AMI
import partition_igraph
from matplotlib import pyplot as plt
from collections import Counter
import random
import pickle
import os
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import roc_curve
from scipy.sparse import csr_matrix
import scipy.sparse as sparse 
import time
from statistics import mode
import seaborn as sns
import omega_index

import sys
sys.path.append('../')
from CAS import *

# DBLP graph

* We load this dataset built from SNAP from a local pickle file with two components:
  * List of edges
  * List of communities

* Build igraph object 'G'

In [2]:
## read data
with open('../Datasets/DBLPFull.pkl', 'rb') as handle:
    b = pickle.load(handle)
Communities = b["Community lists"][:-1]
Edges = b['edges'][:-1]


# number of comms
print('number of GT communities:',len(Communities))

# community sizes
print('most common community sizes:')
print(Counter([len(x) for x in Communities]).most_common(10))

# number of edges
print('\nnumber of edges:',len(Edges))
G = ig.Graph.TupleList([tuple(e) for e in Edges])

# Dictionary: node name to node id
v_dct = {j:i for i,j in enumerate(G.vs['name'])}
print('number of nodes:',G.vcount(),'\n')

# store all communities for each node
for v in G.vs:
    v['comms'] = []
## append communities in 1-based format (we keep 0 for the outliers - no communities)
for i in range(len(Communities)):
    for v in Communities[i]:
        G.vs[v_dct[v]]['comms'].append(i+1)
G.vs['n_comms'] = [len(x) for x in G.vs['comms']]
print('Number of memberships (most frequent):')
print(Counter(G.vs['n_comms']).most_common(10), '\n')

## Add community degrees to G (deg_A(v)'s)
for v in G.vs:
    ctr = Counter([i for x in v.neighbors() for i in x['comms']])
    v['degrees'] = [ctr[i] if i in ctr.keys() else 0 for i in v['comms'] ]

## Add pseudo single community ground truth: pick community with highest dev_A(v) for each v
G.vs['gt'] = [v['comms'][np.argmax(v['degrees'])] if len(v['degrees'])>0 else 0 for v in G.vs]


number of GT communities: 13477
most common community sizes:
[(6, 3680), (7, 2195), (8, 1461), (9, 1005), (10, 747), (11, 564), (12, 391), (13, 310), (14, 244), (16, 204)]

number of edges: 1049866
number of nodes: 317080 

Number of memberships (most frequent):
[(1, 150192), (0, 56082), (2, 43279), (3, 20163), (4, 11770), (5, 7909), (6, 5513), (7, 3915), (8, 3011), (9, 2332)] 



In [3]:
## eta
np.mean([x for x in G.vs['n_comms'] if x>0])


2.7579521682158483

In [4]:
## Compute the external edge fraction given communities for each node
C = Counter([len(set(G.vs[e.source]['comms']).intersection(set(G.vs[e.target]['comms'])))>0 for e in G.es])
print('xi_hat =',C[0]/(C[0]+C[1]))


xi_hat = 0.1088738943827117


## Ego split?

In [5]:
def EgoSplit(G, split='CC', algo='LP'):
    g = G.copy()
    ## implement ego-split approach with LP+LP and LP+ECG
    g.vs['original'] = g.vs['name']
    ## use the vertex names to avoid issues when vertices are re-mapped ...
    names = g.vs['name']
    ## step 1 - ego-net splits
    ctr = 1
    for nm in names:
        if ctr%1000==0:
            print(ctr)
        ctr+=1
        v = g.vs.find(nm).index
        n = g.neighbors(v)
        sg = g.subgraph(n)
        if split == 'LP':
            x = sg.community_label_propagation().membership
        else:
            x = sg.connected_components().membership
        if np.min(x)==-1:
            x = [i+1 for i in x]
        for j in set(x):
            g.add_vertex(name=nm+'.'+str(j),original=nm)

        l = sg.vs['name']
        for j in range(len(x)):
            g.add_edge(nm+'.'+str(x[j]) , l[j])
        g.delete_vertices(v)
    ## step 2 -- cluster w.r.t. multiple personae
    if algo=='LP':
        cl = g.community_label_propagation()
    else:
        cl = g.community_ecg(ens_size=32, final='leiden') ## Leiden
    C = [set(sg.vs['original']) for sg in cl.subgraphs()]
    return C


In [6]:
G.vs['name'] = [str(x) for x in G.vs['name']]


In [7]:
V = np.where(np.array(G.vs['n_comms'])==0)[0]
G.delete_vertices(V)


In [8]:
G = G.connected_components().giant()


In [9]:
with open('es.pkl','rb') as fp:
    ES = pickle.load(fp)


In [10]:
import csv
import subprocess
oNMI = '/Users/francois/Book/GraphMiningNotebooks/oNMI/onmi'          ## overlapping NMI executable
#oNMI = '/work/home/fcthebe/Tools/oNMI/onmi'          ## overlapping NMI executable
def compute_oNMI(First, Second):
    fn1 = '__'+str(random.random())[2:]
    with open(fn1,"w") as f:
        wr = csv.writer(f, delimiter=" ")
        wr.writerows(First)
    f.close()   

    fn2 = '__'+str(random.random())[2:]
    with open(fn2,"w") as f:
        wr = csv.writer(f, delimiter=" ")
        wr.writerows(Second)
    f.close()   
    x = float(subprocess.check_output([oNMI,fn1,fn2]).decode("utf-8").split()[1])
    _ = os.system('rm '+fn1)
    _ = os.system('rm '+fn2)
    return x


In [11]:
min_size = 5
ego = [list(x) for x in ES if len(x)>=min_size]    


In [12]:
Nodes = set(G.vs['name'])


In [13]:
GT = []
for c in Communities:
    x = [str(x) for x in c if str(x) in Nodes]
    if len(x)>0:
        GT.append(x)    


In [14]:
compute_oNMI(ego,GT)


0.047111

In [33]:
v_dct = {j:i for i,j in enumerate(G.vs['name'])}


In [35]:
C = sparse.csc_matrix((G.vcount(),len(ego)))
C.indices = np.array([v_dct[i] for j in ego for i in j])
C.data = np.repeat(1,len(C.indices))
ptr = [0]
ctr = 0
for x in ego:
    ctr += len(x)
    ptr.append(ctr)
C.indptr = np.array(ptr)
M = C.tocsr()


In [16]:
def memberships2list(S):
    L = []
    for i in range(len(S.indptr)-1):
        if S.indptr[i] == S.indptr[i+1]:
            L.append([0]) ## no membership == outlier (community 0)
        else:
            L.append(list(S.indices[S.indptr[i]:S.indptr[i+1]]+1)) ## 1-based
    return L
def mem2comms(X):
    nc = max(set([i for j in X for i in j]))+1  
    n = len(X)
    L = [[] for _ in range(nc)]
    for i in range(n):
        for j in X[i]:
            L[j].append(i)
    return L


In [45]:
IEF, Beta, C, Pv, DegPart = CAS(G.get_adjacency_sparse(), M, alpha=1)
for th in [.05,.1,.15,.2,.25,.3]:
    L = []
    for i in range(Beta.shape[1]):
        x = np.where( (np.array(Beta[:,i].todense()).flatten() >= th))[0]
        if len(x)>0:
            L.append([G.vs[i]['name'] for i in list(x)])    
    print(th,compute_oNMI(GT,L))
    

0.05 0.0207908
0.1 0.0288185
0.15 0.0350697
0.2 0.0375962
0.25 0.0385652
0.3 0.0374755
